<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include <cmath>
#include <memory>
#include "mindspore/lite/schema/inner/model_generated.h"
#include "mindspore/lite/include/model.h"
#include "common/common_test.h"
#include "include/lite_session.h"
#include "include/context.h"
#include "include/errorcode.h"
#include "mindspore/core/utils/log_adapter.h"
#include "tools/converter/model_parser.h"
#include "tools/converter/anf_transform.h"
#include "src/common/anf_exporter/anf_exporter.h"

namespace mindspore {
class ConvBiasAddFusionTest : public mindspore::Common {
 public:
  ConvBiasAddFusionTest() = default;
};
using MetaGraphTptr = std::shared_ptr<schema::MetaGraphT>;
using CNodeTptr = std::unique_ptr<schema::CNodeT>;

namespace {
CNodeTptr BuildConv2D() {
  auto convNode = std::make_unique<schema::CNodeT>();
  convNode->inputIndex = {0, 1};
  convNode->outputIndex = {2};
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_Conv2D;
  auto prim1 = new schema::Conv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelOut = 3;
  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}
CNodeTptr BuildDepthwiseConv2D() {
  auto convNode = std::make_unique<schema::CNodeT>();
  convNode->inputIndex = {0, 1};
  convNode->outputIndex = {2};
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_DepthwiseConv2D;
  auto prim1 = new schema::DepthwiseConv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelIn = 1;
  prim1->channelMultiplier = 3;

  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}

MetaGraphTptr BuildGraph(schema::PrimitiveType conv_type,
                         schema::PrimitiveType add_type) {
  auto meta_graph = std::make_shared<schema::MetaGraphT>();
  meta_graph->name = "graph";
  // conv node
  CNodeTptr convNode;
  if (conv_type == schema::PrimitiveType_Conv2D) {
    convNode = BuildConv2D();
  } else {
    convNode = BuildDepthwiseConv2D();
  };

  meta_graph->nodes.emplace_back(std::move(convNode));

  // biasadd node
  auto biasadd_node = std::make_unique<schema::CNodeT>();
  biasadd_node->inputIndex = {2, 3};
  biasadd_node->outputIndex = {4};
  biasadd_node->primitive = std::make_unique<schema::PrimitiveT>();
  biasadd_node->primitive->value.type = add_type;
  auto prim2 = new schema::BiasAddT;
  biasadd_node->primitive->value.value = prim2;
  biasadd_node->name = "BiasAdd";
  meta_graph->nodes.emplace_back(std::move(biasadd_node));

  meta_graph->inputIndex = {0};
  meta_graph->outputIndex = {4};

  // input 0: data
  auto input0 = std::make_unique<schema::TensorT>();
  input0->nodeType = schema::NodeType::NodeType_ValueNode;
  input0->format = schema::Format_NHWC;
  input0->dataType = TypeId::kNumberTypeFloat32;
  input0->dims = {1, 5, 5, 3};
  input0->offset = -1;
  meta_graph->allTensors.emplace_back(std::move(input0));

  //input 1: weight
  auto input1 = std::make_unique<schema::TensorT>();
  input1->nodeType = schema::NodeType::NodeType_ValueNode;
  input1->format = schema::Format_KHWC;
  input1->dataType = TypeId::kNumberTypeFloat32;
  input1->dims = {8, 3, 3, 3};
  input1->data.resize(sizeof(float) * 8 * 3 * 3 * 3);
  meta_graph->allTensors.emplace_back(std::move(input1));

  // conv output
  auto conv_output = std::make_unique<schema::TensorT>();
  conv_output->nodeType = schema::NodeType::NodeType_Parameter;
  conv_output->format = schema::Format_NHWC;
  conv_output->dataType = TypeId::kNumberTypeFloat32;
  conv_output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(conv_output));

  // input2: bias
  auto input2 = std::make_unique<schema::TensorT>();
  input2->nodeType = schema::NodeType::NodeType_ValueNode;
  input2->format = schema::Format_NHWC;
  input2->dataType = TypeId::kNumberTypeFloat32;
  input2->dims = {1, 5, 5, 8};
  input2->data.resize(sizeof(float) * 8 * 5 * 5);  //here
  meta_graph->allTensors.emplace_back(std::move(input2));

  // final output
  auto output = std::make_unique<schema::TensorT>();
  output->nodeType = schema::NodeType::NodeType_Parameter;
  output->format = schema::Format_NHWC;
  output->dataType = TypeId::kNumberTypeFloat32;
  output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(output));
  return meta_graph;
}
}
TEST_F(ConvBiasAddFusionTest, TestConvAddNode) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_Conv2D, schema::PrimitiveType_BiasAdd);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsConv2D()->hasBias, true);
  }
  MS_LOG(INFO) << "Passed";
}

TEST_F(ConvBiasAddFusionTest, TestDeptiwiseConvAddNode) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_DepthwiseConv2D, schema::PrimitiveType_Add);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsDepthwiseConv2D()->hasBias, true);
  }
  MS_LOG(INFO) << "Passed";
}

TEST_F(ConvBiasAddFusionTest, TestBadCase_ConvAdd) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_DepthwiseConv2D, schema::PrimitiveType_MatMul);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 2);
  for (auto &cnode:new_meta_graph->nodes) {
    if (cnode->primitive->value.type == schema::PrimitiveType_DepthwiseConv2D) {
      ASSERT_EQ(cnode->primitive->value.AsDepthwiseConv2D()->hasBias, false);
    }
  }
  MS_LOG(INFO) << "Passed";
}

}  // namespace mindspore


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include <cmath>
#include <memory>
#include "mindspore/lite/schema/inner/model_generated.h"
#include "mindspore/lite/include/model.h"
#include "common/common_test.h"
#include "include/lite_session.h"
#include "include/context.h"
#include "include/errorcode.h"
#include "mindspore/core/utils/log_adapter.h"
#include "tools/converter/model_parser.h"
#include "tools/converter/anf_transform.h"
#include "src/common/anf_exporter/anf_exporter.h"

namespace mindspore {
class ConvActivationFusionTest : public mindspore::Common {
 public:
  ConvActivationFusionTest() = default;
};
using MetaGraphTptr = std::shared_ptr<schema::MetaGraphT>;
using CNodeTptr = std::unique_ptr<schema::CNodeT>;

namespace {
CNodeTptr BuildConv2D() {
  auto convNode = std::make_unique<schema::CNodeT>();
  convNode->inputIndex = {0, 1};
  convNode->outputIndex = {2};
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_Conv2D;
  auto prim1 = new schema::Conv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelOut = 3;
  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}
CNodeTptr BuildDepthwiseConv2D() {
  auto convNode = std::make_unique<schema::CNodeT>();
  convNode->inputIndex = {0, 1};
  convNode->outputIndex = {2};
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_DepthwiseConv2D;
  auto prim1 = new schema::DepthwiseConv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelIn = 1;
  prim1->channelMultiplier = 3;

  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}

MetaGraphTptr BuildGraph(schema::PrimitiveType conv_type,
                         schema::ActivationType activation_type) {
  auto meta_graph = std::make_shared<schema::MetaGraphT>();
  meta_graph->name = "graph";
  // conv node
  CNodeTptr convNode;
  if (conv_type == schema::PrimitiveType_Conv2D) {
    convNode = BuildConv2D();
  } else {
    convNode = BuildDepthwiseConv2D();
  }
  meta_graph->nodes.emplace_back(std::move(convNode));

  // relu node
  auto next_node = std::make_unique<schema::CNodeT>();
  next_node->inputIndex = {2};
  next_node->outputIndex = {3};
  next_node->primitive = std::make_unique<schema::PrimitiveT>();
  next_node->primitive->value.type = schema::PrimitiveType_Activation;
  auto prim2 = new schema::ActivationT;
  prim2->type = activation_type;
  next_node->primitive->value.value = prim2;
  next_node->name = "activation";
  meta_graph->nodes.emplace_back(std::move(next_node));

  meta_graph->inputIndex = {0};
  meta_graph->outputIndex = {3};

  // input 0: data
  auto input0 = std::make_unique<schema::TensorT>();
  input0->nodeType = schema::NodeType::NodeType_ValueNode;
  input0->format = schema::Format_NHWC;
  input0->dataType = TypeId::kNumberTypeFloat32;
  input0->dims = {1, 5, 5, 3};
  input0->offset = -1;
  meta_graph->allTensors.emplace_back(std::move(input0));

  //input 1: weight
  auto input1 = std::make_unique<schema::TensorT>();
  input1->nodeType = schema::NodeType::NodeType_ValueNode;
  input1->format = schema::Format_KHWC;
  input1->dataType = TypeId::kNumberTypeFloat32;
  input1->dims = {8, 3, 3, 3};
  input1->data.resize(sizeof(float) * 8 * 3 * 3 * 3);
  meta_graph->allTensors.emplace_back(std::move(input1));

  // conv output
  auto conv_output = std::make_unique<schema::TensorT>();
  conv_output->nodeType = schema::NodeType::NodeType_Parameter;
  conv_output->format = schema::Format_NHWC;
  conv_output->dataType = TypeId::kNumberTypeFloat32;
  conv_output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(conv_output));

  // final output
  auto output = std::make_unique<schema::TensorT>();
  output->nodeType = schema::NodeType::NodeType_Parameter;
  output->format = schema::Format_NHWC;
  output->dataType = TypeId::kNumberTypeFloat32;
  output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(output));
  return meta_graph;
}
}
TEST_F(ConvActivationFusionTest, TestConvReluNode) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_Conv2D, schema::ActivationType_RELU);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsConv2D()->activationType, schema::ActivationType_RELU);
  }
}

TEST_F(ConvActivationFusionTest, TestConvRelu6Node) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_Conv2D, schema::ActivationType_RELU6);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsConv2D()->activationType, schema::ActivationType_RELU6);
  }
}

TEST_F(ConvActivationFusionTest, TestBadCase_ConvRelu) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_DepthwiseConv2D, schema::ActivationType_LEAKY_RELU);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 2);
  for (auto &cnode:new_meta_graph->nodes) {
    if (cnode->primitive->value.type == schema::PrimitiveType_DepthwiseConv2D) {
      ASSERT_EQ(cnode->primitive->value.AsDepthwiseConv2D()->activationType, schema::ActivationType_NO_ACTIVATION);
    }
  }
}

}  // namespace mindspore


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include <cmath>
#include <memory>
#include "mindspore/lite/schema/inner/model_generated.h"
#include "mindspore/lite/include/model.h"
#include "common/common_test.h"
#include "include/lite_session.h"
#include "include/context.h"
#include "include/errorcode.h"
#include "mindspore/core/utils/log_adapter.h"
#include "tools/converter/model_parser.h"
#include "tools/converter/anf_transform.h"
#include "src/common/anf_exporter/anf_exporter.h"

namespace mindspore {
class ConvBNFusionTest : public mindspore::Common {
 public:
  ConvBNFusionTest() = default;
};
using MetaGraphTptr = std::shared_ptr<schema::MetaGraphT>;
using CNodeTptr = std::unique_ptr<schema::CNodeT>;

namespace {
CNodeTptr BuildConv2D() {
  auto convNode = std::make_unique<schema::CNodeT>();
  convNode->inputIndex = {0, 1};
  convNode->outputIndex = {2};
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_Conv2D;
  auto prim1 = new schema::Conv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelOut = 3;
  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}
CNodeTptr BuildDepthwiseConv2D() {
  auto convNode = std::make_unique<schema::CNodeT>();
  convNode->inputIndex = {0,1,2};
  convNode->outputIndex = {3};
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_DepthwiseConv2D;
  auto prim1 = new schema::DepthwiseConv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelIn = 1;
  prim1->channelMultiplier = 3;

  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}
// caffe bn op has 3 inputs
MetaGraphTptr BuildCaffeGraph(schema::PrimitiveType conv_type) {
  auto meta_graph = std::make_shared<schema::MetaGraphT>();
  meta_graph->name = "graph";
  // conv node
  CNodeTptr convNode;
  if (conv_type == schema::PrimitiveType_Conv2D) {
    convNode = BuildConv2D();
  } else {
    convNode = BuildDepthwiseConv2D();
  };

  meta_graph->nodes.emplace_back(std::move(convNode));

  // bn_node
  auto bn_node = std::make_unique<schema::CNodeT>();
  bn_node->inputIndex = {2, 3,4};
  bn_node->outputIndex = {5};
  bn_node->primitive = std::make_unique<schema::PrimitiveT>();
  bn_node->primitive->value.type = schema::PrimitiveType_CaffeBatchNorm;
  auto prim2 = new schema::CaffeBatchNormT;
  bn_node->primitive->value.value = prim2;
  bn_node->name = "bn";
  meta_graph->nodes.emplace_back(std::move(bn_node));

  // input 0: data
  auto input0 = std::make_unique<schema::TensorT>();
  input0->nodeType = schema::NodeType::NodeType_ValueNode;
  input0->format = schema::Format_NHWC;
  input0->dataType = TypeId::kNumberTypeFloat32;
  input0->dims = {1, 5, 5, 3};
  input0->offset = -1;
  meta_graph->allTensors.emplace_back(std::move(input0));

  //input 1: weight
  auto input1 = std::make_unique<schema::TensorT>();
  input1->nodeType = schema::NodeType::NodeType_ValueNode;
  input1->format = schema::Format_KHWC;
  input1->dataType = TypeId::kNumberTypeFloat32;
  input1->dims = {8, 3, 3, 3};
  input1->data.resize(sizeof(float) * 8 * 3 * 3 * 3);
  meta_graph->allTensors.emplace_back(std::move(input1));

  // conv output
  auto conv_output = std::make_unique<schema::TensorT>();
  conv_output->nodeType = schema::NodeType::NodeType_Parameter;
  conv_output->format = schema::Format_NHWC;
  conv_output->dataType = TypeId::kNumberTypeFloat32;
  conv_output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(conv_output));

  //caffe bn : mean
  auto input2 = std::make_unique<schema::TensorT>();
  input2->nodeType = schema::NodeType::NodeType_ValueNode;
  input2->format = schema::Format_NHWC;
  input2->dataType = TypeId::kNumberTypeFloat32;
  input2->dims = {1, 5, 5, 8};
  input2->data.resize(sizeof(float) * 8 * 5 * 5);  //here
  meta_graph->allTensors.emplace_back(std::move(input2));

  //caffe bn : var
  auto input3 = std::make_unique<schema::TensorT>();
  input3->nodeType = schema::NodeType::NodeType_ValueNode;
  input3->format = schema::Format_NHWC;
  input3->dataType = TypeId::kNumberTypeFloat32;
  input3->dims = {1, 5, 5, 8};
  input3->data.resize(sizeof(float) * 8 * 5 * 5);  //here
  meta_graph->allTensors.emplace_back(std::move(input3));


  // final bn output
  auto output = std::make_unique<schema::TensorT>();
  output->nodeType = schema::NodeType::NodeType_Parameter;
  output->format = schema::Format_NHWC;
  output->dataType = TypeId::kNumberTypeFloat32;
  output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(output));
  meta_graph->inputIndex = {0};
  meta_graph->outputIndex = {5};
  return meta_graph;
}

// tf bn op has 4 inputs
MetaGraphTptr BuildTFGraph(schema::PrimitiveType conv_type) {
  auto meta_graph = std::make_shared<schema::MetaGraphT>();
  meta_graph->name = "graph";
  // conv node
  CNodeTptr convNode;
  if (conv_type == schema::PrimitiveType_Conv2D) {
    convNode = BuildConv2D();
  } else {
    convNode = BuildDepthwiseConv2D();
  };

  meta_graph->nodes.emplace_back(std::move(convNode));

  // bn_node
  auto bn_node = std::make_unique<schema::CNodeT>();
  bn_node->inputIndex = {3,4,5,6,7};
  bn_node->outputIndex = {8};
  bn_node->primitive = std::make_unique<schema::PrimitiveT>();
  bn_node->primitive->value.type = schema::PrimitiveType_FusedBatchNorm;
  auto prim2 = new schema::FusedBatchNormT;
  bn_node->primitive->value.value = prim2;
  bn_node->name = "bn";
  meta_graph->nodes.emplace_back(std::move(bn_node));

  // input 0: data
  auto input0 = std::make_unique<schema::TensorT>();
  input0->nodeType = schema::NodeType::NodeType_ValueNode;
  input0->format = schema::Format_NHWC;
  input0->dataType = TypeId::kNumberTypeFloat32;
  input0->dims = {1, 5, 5, 3};
  input0->offset = -1;
  meta_graph->allTensors.emplace_back(std::move(input0));


  //input 1: conv_bias
  auto input11 = std::make_unique<schema::TensorT>();
  input11->nodeType = schema::NodeType::NodeType_ValueNode;
  input11->format = schema::Format_KHWC;
  input11->dataType = TypeId::kNumberTypeFloat32;
  input11->dims = {8, 3, 3, 3};
  input11->data.resize(sizeof(float) * 8 * 3 * 3 * 3);
  meta_graph->allTensors.emplace_back(std::move(input11));

  //input 1: weight
  auto input1 = std::make_unique<schema::TensorT>();
  input1->nodeType = schema::NodeType::NodeType_ValueNode;
  input1->format = schema::Format_KHWC;
  input1->dataType = TypeId::kNumberTypeFloat32;
  input1->dims = {8, 3, 3, 3};
  input1->data.resize(sizeof(float) * 8 * 3 * 3 * 3);
  meta_graph->allTensors.emplace_back(std::move(input1));

  // conv output
  auto conv_output = std::make_unique<schema::TensorT>();
  conv_output->nodeType = schema::NodeType::NodeType_Parameter;
  conv_output->format = schema::Format_NHWC;
  conv_output->dataType = TypeId::kNumberTypeFloat32;
  conv_output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(conv_output));

  //tflite bn : scale
  auto input2 = std::make_unique<schema::TensorT>();
  input2->nodeType = schema::NodeType::NodeType_ValueNode;
  input2->format = schema::Format_NHWC;
  input2->dataType = TypeId::kNumberTypeFloat32;
  input2->dims = {1, 5, 5, 8};
  input2->data.resize(sizeof(float) * 8 * 5 * 5);
  meta_graph->allTensors.emplace_back(std::move(input2));

  //tflite bn : bias
  auto input3 = std::make_unique<schema::TensorT>();
  input3->nodeType = schema::NodeType::NodeType_ValueNode;
  input3->format = schema::Format_NHWC;
  input3->dataType = TypeId::kNumberTypeFloat32;
  input3->dims = {1, 5, 5, 8};
  input3->data.resize(sizeof(float) * 8 * 5 * 5);
  meta_graph->allTensors.emplace_back(std::move(input3));

  //tflite bn : mean
  auto input4 = std::make_unique<schema::TensorT>();
  input4->nodeType = schema::NodeType::NodeType_ValueNode;
  input4->format = schema::Format_NHWC;
  input4->dataType = TypeId::kNumberTypeFloat32;
  input4->dims = {1, 5, 5, 8};
  input4->data.resize(sizeof(float) * 8 * 5 * 5);
  meta_graph->allTensors.emplace_back(std::move(input4));

  //tflite bn : var
  auto input5 = std::make_unique<schema::TensorT>();
  input5->nodeType = schema::NodeType::NodeType_ValueNode;
  input5->format = schema::Format_NHWC;
  input5->dataType = TypeId::kNumberTypeFloat32;
  input5->dims = {1, 5, 5, 8};
  input5->data.resize(sizeof(float) * 8 * 5 * 5);
  meta_graph->allTensors.emplace_back(std::move(input5));

  // final output
  auto output = std::make_unique<schema::TensorT>();
  output->nodeType = schema::NodeType::NodeType_Parameter;
  output->format = schema::Format_NHWC;
  output->dataType = TypeId::kNumberTypeFloat32;
  output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(output));
  meta_graph->inputIndex = {0};
  meta_graph->outputIndex = {8};
  return meta_graph;
}

}
TEST_F(ConvBNFusionTest, TestConvAddNode) {
  auto meta_graph = BuildCaffeGraph(schema::PrimitiveType_Conv2D);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsConv2D()->hasBias, true);
  }
}

TEST_F(ConvBNFusionTest, TestDeptiwiseConvAddNode) {
  auto meta_graph = BuildTFGraph(schema::PrimitiveType_DepthwiseConv2D);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsDepthwiseConv2D()->hasBias, true);
  }
}

}  // namespace mindspore


In [ ]:
/**
 * Copyright 2020 Huawei Technologies Co., Ltd
 *
 * Licensed under the Apache License, Version 2.0 (the "License");
 * you may not use this file except in compliance with the License.
 * You may obtain a copy of the License at
 *
 * http://www.apache.org/licenses/LICENSE-2.0
 *
 * Unless required by applicable law or agreed to in writing, software
 * distributed under the License is distributed on an "AS IS" BASIS,
 * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
 * See the License for the specific language governing permissions and
 * limitations under the License.
 */

#include <cmath>
#include <memory>
#include "mindspore/lite/schema/inner/model_generated.h"
#include "mindspore/lite/include/model.h"
#include "common/common_test.h"
#include "include/lite_session.h"
#include "include/context.h"
#include "include/errorcode.h"
#include "mindspore/core/utils/log_adapter.h"
#include "tools/converter/model_parser.h"
#include "tools/converter/anf_transform.h"
#include "src/common/anf_exporter/anf_exporter.h"

namespace mindspore {
class ConvScaleFusionTest : public mindspore::Common {
 public:
  ConvScaleFusionTest() = default;
};
using MetaGraphTptr = std::shared_ptr<schema::MetaGraphT>;
using CNodeTptr = std::unique_ptr<schema::CNodeT>;

namespace {
//conv 2d has 2 inputs
CNodeTptr BuildConv2D(int with_bias_flag) {
  auto convNode = std::make_unique<schema::CNodeT>();
  if (with_bias_flag) {
    convNode->inputIndex = {0, 1, 2};
    convNode->outputIndex = {3};
  } else {
    convNode->inputIndex = {0, 1};
    convNode->outputIndex = {2};
  }
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_Conv2D;
  auto prim1 = new schema::Conv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelOut = 3;
  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}
// conv2d has 3 inputs
CNodeTptr BuildDepthwiseConv2D(int with_bias_flag) {
  auto convNode = std::make_unique<schema::CNodeT>();
  if (with_bias_flag) {
    convNode->inputIndex = {0, 1, 2};
    convNode->outputIndex = {3};
  } else {
    convNode->inputIndex = {0, 1};
    convNode->outputIndex = {2};
  }
  convNode->primitive = std::make_unique<schema::PrimitiveT>();
  convNode->primitive->value.type = schema::PrimitiveType_DepthwiseConv2D;
  auto prim1 = new schema::DepthwiseConv2DT;
  prim1->padMode = schema::PadMode_SAME;
  //prim1->channelIn = 1;
  //prim1->channelOut = 8;
  prim1->format = schema::Format_NHWC;
  prim1->strideH = 1;
  prim1->strideW = 1;
  prim1->kernelH = 3;
  prim1->kernelW = 3;
  prim1->dilateH = 1;
  prim1->dilateW = 1;
  prim1->channelIn = 1;
  prim1->channelMultiplier = 3;

  convNode->primitive->value.value = prim1;
  convNode->name = "Conv2D";
  return convNode;
}

MetaGraphTptr BuildGraph(schema::PrimitiveType conv_type,bool conv_with_bias) {
  auto meta_graph = std::make_shared<schema::MetaGraphT>();
  meta_graph->name = "graph";
  // conv node
  CNodeTptr convNode;
  if (conv_type == schema::PrimitiveType_Conv2D) {
    convNode = BuildConv2D(conv_with_bias);
  } else {
    convNode = BuildDepthwiseConv2D(conv_with_bias);
  };

  meta_graph->nodes.emplace_back(std::move(convNode));

  // scale_node weight bias
  auto scale_node = std::make_unique<schema::CNodeT>();
  if (conv_with_bias) {
    scale_node->inputIndex = {3, 4, 5};
    scale_node->outputIndex = {6};
  } else {
    scale_node->inputIndex = {2, 3, 4};
    scale_node->outputIndex = {5};
  }

  scale_node->primitive = std::make_unique<schema::PrimitiveT>();
  scale_node->primitive->value.type = schema::PrimitiveType_Scale;
  auto prim2 = new schema::ScaleT;
  scale_node->primitive->value.value = prim2;
  scale_node->name = "scale";
  meta_graph->nodes.emplace_back(std::move(scale_node));

  // input 0: data
  auto input0 = std::make_unique<schema::TensorT>();
  input0->nodeType = schema::NodeType::NodeType_ValueNode;
  input0->format = schema::Format_NHWC;
  input0->dataType = TypeId::kNumberTypeFloat32;
  input0->dims = {1, 5, 5, 3};
  input0->offset = -1;
  meta_graph->allTensors.emplace_back(std::move(input0));


  //input 1: weight
  auto input1 = std::make_unique<schema::TensorT>();
  input1->nodeType = schema::NodeType::NodeType_ValueNode;
  input1->format = schema::Format_KHWC;
  input1->dataType = TypeId::kNumberTypeFloat32;
  input1->dims = {8, 3, 3, 3};
  input1->data.resize(sizeof(float) * 8 * 3 * 3 * 3);
  meta_graph->allTensors.emplace_back(std::move(input1));

  if (conv_with_bias) {
    // input 00: bias
    auto input00 = std::make_unique<schema::TensorT>();
    input00->nodeType = schema::NodeType::NodeType_ValueNode;
    input00->format = schema::Format_NHWC;
    input00->dataType = TypeId::kNumberTypeFloat32;
    input00->dims = {1, 5, 5, 3};
    input00->offset = -1;
    meta_graph->allTensors.emplace_back(std::move(input00));
  }

  // conv output
  auto conv_output = std::make_unique<schema::TensorT>();
  conv_output->nodeType = schema::NodeType::NodeType_Parameter;
  conv_output->format = schema::Format_NHWC;
  conv_output->dataType = TypeId::kNumberTypeFloat32;
  conv_output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(conv_output));

  //scale weight input
  auto input2 = std::make_unique<schema::TensorT>();
  input2->nodeType = schema::NodeType::NodeType_ValueNode;
  input2->format = schema::Format_NHWC;
  input2->dataType = TypeId::kNumberTypeFloat32;
  input2->dims = {1, 5, 5, 8};
  input2->data.resize(sizeof(float) * 8 * 5 * 5);  //here
  meta_graph->allTensors.emplace_back(std::move(input2));

  //scale bias input
  auto input3 = std::make_unique<schema::TensorT>();
  input3->nodeType = schema::NodeType::NodeType_ValueNode;
  input3->format = schema::Format_NHWC;
  input3->dataType = TypeId::kNumberTypeFloat32;
  input3->dims = {1, 5, 5, 8};
  input3->data.resize(sizeof(float) * 8 * 5 * 5);  //here
  meta_graph->allTensors.emplace_back(std::move(input3));


  // final scale output
  auto output = std::make_unique<schema::TensorT>();
  output->nodeType = schema::NodeType::NodeType_Parameter;
  output->format = schema::Format_NHWC;
  output->dataType = TypeId::kNumberTypeFloat32;
  output->dims = {1, 5, 5, 8};
  meta_graph->allTensors.emplace_back(std::move(output));
  if(conv_with_bias){
    meta_graph->inputIndex = {0};
    meta_graph->outputIndex = {6};
  }else{
    meta_graph->inputIndex = {0};
    meta_graph->outputIndex = {5};
  }

  return meta_graph;
}

}
TEST_F(ConvScaleFusionTest, TestConvScaleNode) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_Conv2D,true);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsConv2D()->hasBias, true);
  }
}

TEST_F(ConvScaleFusionTest, TestDeptiwiseConvScaleNode) {
  auto meta_graph = BuildGraph(schema::PrimitiveType_DepthwiseConv2D,false);
  auto func_graph = lite::ModelParser::Fb2Anf(meta_graph.get());
  auto anf_transform = new lite::AnfTransform();
  auto new_graph = anf_transform->Transform(func_graph);
  ASSERT_NE(nullptr, new_graph);
  auto new_meta_graph = lite::Export(new_graph);
  ASSERT_EQ(new_meta_graph->nodes.size(), 1);
  for (auto &cnode:new_meta_graph->nodes) {
    ASSERT_EQ(cnode->primitive->value.AsDepthwiseConv2D()->hasBias, true);
    ASSERT_EQ(cnode->inputIndex.size(),3);
  }
}

}  // namespace mindspore


#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

training